In [16]:
import pandas as pd
import torch
import os
import sklearn
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import sys
sys.path.append('./..')
sys.path.append('./../..')
from pandarallel import pandarallel
pandarallel.initialize()
import glob 
import yaml
import pickle
from scipy.special import softmax as SOFTMAX
import matplotlib.pyplot as plt
from sklearn.metrics import auc
try:
    from common_utils import AD_result_fetcher
except:
    from .common_utils import AD_result_fetcher
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from time import time
from matplotlib import pyplot as plt
from sklearn.metrics import silhouette_score

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [17]:
config_file = 'config.yaml'
with open(config_file,'r') as fh:
    CONFIG = yaml.safe_load(fh)



In [18]:
DIR = 'us_import1'

In [19]:
labelled_results = AD_result_fetcher.read_in_AD_result(DIR)
score_threshold = np.percentile(labelled_results['score'],CONFIG['ANOMALY_PERCENTILE_THRESHOLD'])
print(CONFIG['ANOMALY_PERCENTILE_THRESHOLD'], score_threshold)

22648 22648
95 0.5650949312585849


In [22]:
def obtain_bipartite_embeddings():
    global DIR
    LOC = './saved_model_data'
    files = sorted(glob.glob(os.path.join(LOC,DIR, '**.npy')))
    print(files)

    
obtain_bipartite_embeddings()

['./saved_model_data/us_import1/ConsigneePanjivaID_gnn_32.npy', './saved_model_data/us_import1/ShipperPanjivaID_gnn_32.npy']


In [23]:
labelled_results.head(10)

,PanjivaRecordID,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID,label,score
670,109851578002121001,428,3164,4932,5062,5285,5354,5425,11092,1,1.000000
681,106832265002231001,225,2101,5005,5072,5295,5387,5434,8701,1,0.998987
266,1079874181001,219,4446,5004,5101,5297,5319,5453,6449,-1,0.993772
388,1143245431001,225,1748,4993,5072,5295,5387,5434,10337,-1,0.993298
158,1152925491001,215,2551,4898,5087,5302,5395,5432,7610,-1,0.991523
797,108424326003391001,148,1748,4898,5087,5265,5395,5434,10760,1,0.986588
411,1154324571001,428,1474,4932,5124,5289,5354,5425,9520,-1,0.982792
718,108321141002601001,303,1748,4996,5046,5295,5329,5434,9839,1,0.981981
784,109628827003261001,303,1748,4996,5046,5295,5329,5434,9839,1,0.981981
245,1072418701001,225,2079,5004,5072,5295,5387,5434,6574,-1,0.975518
